<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/using-vectara-with-llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectara and LlamaIndex

In this notebook we are going to show how to use Vectara with LlamaIndex.

Please note: We have updated our integration with LlamaIndex (llama-index-indices-managed-vectara version >= 0.4.0) to use our API v2, so some of the parameters and credentials required for our integration have changed. The latest changes are reflected in this example notebook.

## About Vectara

[Vectara](https://vectara.com/) is the trusted AI Assistant and Agent platform which focuses on enterprise readiness for mission-critical applications. 

Vectara provides an end-to-end managed service for Retrieval Augmented Generation or [RAG](https://vectara.com/grounded-generation/), which includes:

1. An integrated API for processing input data, including text extraction from documents and ML-based chunking.

2. The state-of-the-art [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) embeddings model. Each text chunk is encoded into a vector embedding using Boomerang, and stored in the Vectara internal knowledge (vector+text) store. Thus, when using Vectara with LlamaIndex you do not need to call a separate embedding model - this happens automatically within the Vectara backend.

3. A query service that automatically encodes the query into embeddings and retrieves the most relevant text segmentsthrough [hybrid search](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching) and a variety of [reranking](https://docs.vectara.com/docs/api-reference/search-apis/reranking) strategies, including a [multilingual reranker](https://docs.vectara.com/docs/learn/vectara-multi-lingual-reranker), [maximal marginal relevance (MMR) reranker](https://docs.vectara.com/docs/learn/mmr-reranker), [user-defined function reranker](https://docs.vectara.com/docs/learn/user-defined-function-reranker), and a [chain reranker](https://docs.vectara.com/docs/learn/chain-reranker) that provides a way to chain together multiple reranking methods to achieve better control over the reranking, combining the strengths of various reranking methods.

4. An option to create a [generative summary](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview) with a wide selection of LLM summarizers (including Vectara's [Mockingbird](https://vectara.com/blog/mockingbird-is-a-rag-specific-llm-that-beats-gpt-4-gemini-1-5-pro-in-rag-output-quality/), trained specifically for RAG-based tasks), based on the retrieved documents, including citations.

See the [Vectara API documentation](https://docs.vectara.com/docs/) for more information on how to use the API.

The main benefits of using Vectara RAG-as-a-service to build your application are:
* **Accuracy and Quality**: Vectara provides an end-to-end platform that focuses on eliminating hallucinations, reducing bias, and safeguarding copyright integrity.
* **Security**: Vectara's platform provides acess control--protecting against prompt injection attacks--and meets SOC2 and HIPAA compliance.
* **Explainability**: Vectara makes it easy to troubleshoot bad results by clearly explaining rephrased queries, LLM prompts, retrieved results, and agent actions.

## About LlamaIndex

LlamaIndex is a "data framework" to help you build LLM apps:

1. It includes **data connectors** to ingest your existing data sources and data formats (APIs, PDFs, docs, SQL, etc.)
2. It provides ways to **structure your data** (indices, graphs) so that this data can be easily used with LLMs.
3. It provides an **advanced retrieval/query interface over your data**: Feed in any LLM input prompt, get back retrieved context and knowledge-augmented output.

LlamaIndex's high-level API allows beginner users to use LlamaIndex to ingest and query their data in just a few lines of code, whereas its lower-level APIs allow advanced users to customize and extend any module (data connectors, indices, retrievers, query engines, reranking modules), to fit their needs.

Vectara is implemented in LlamaIndex as a [Managed Service](https://docs.llamaindex.ai/en/stable/community/integrations/managed_indices.html#vectara), abstracting all of Vectara's powerful API so they are easily integrated into LlamaIndex.

In this notebook, we will demonstrate some of the great ways you can use Vectara together with LlamaIndex.

## Getting Started

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

To get started with Vectara, [sign up](https://console.vectara.com/signup?utm_source=vectara&utm_medium=signup&utm_term=DevRel&utm_content=example-notebooks&utm_campaign=vectara-signup-DevRel-example-notebooks) (if you haven't already) and follow our [quickstart](https://docs.vectara.com/docs/quickstart) guide to create a corpus and an API key. 

Once you have these, you can provide them as environment variables or explicitly define them in the code cell below. These will be used by the LlamaIndex code later on to connect the index to your Vectara corpus:

In [1]:
# !pip install -U llama-index llama-index-indices-managed-vectara arxiv

import os
# os.environ['VECTARA_API_KEY'] = "<YOUR_VECTARA_API_KEY>"
# os.environ['VECTARA_CORPUS_KEY'] = "<YOUR_VECTARA_CORPUS_KEY>"

## Loading Data Into Vectara

As mentioned above, Vectara is a RAG managed service, and in many cases data may be uploaded to the index ahead of time (e.g. by using [Airbyte](https://docs.airbyte.com/integrations/destinations/vectara), directly via Vectara's [indexing API](https://docs.vectara.com/docs/api-reference/indexing-apis/indexing) or using tools like [vectara-ingest](https://github.com/vectara/vectara-ingest)), but another easy way is via the VectaraIndex constructor: `from_documents()`.

For this notebook, we will assume the Vectara corpus is empty and will load PDF documents from Arxiv, using Python's [arxiv](https://github.com/lukasschwab/arxiv.py) library. We will pull in data from the top papers related to "embedding model":

In [2]:
import arxiv

client = arxiv.Client()
search = arxiv.Search(
  query = "(ti:embedding model) OR (ti:sentence embedding)",
  max_results = 100,
  sort_by = arxiv.SortCriterion.Relevance
)
papers = list(client.results(search))

In [3]:
[p.entry_id for p in papers][:5]

['http://arxiv.org/abs/2402.14776v3',
 'http://arxiv.org/abs/2007.01852v2',
 'http://arxiv.org/abs/1910.13291v1',
 'http://arxiv.org/abs/2104.06719v1',
 'http://arxiv.org/abs/1511.08198v3']

Next, download the Arxiv paper, and upload them into Vectara using the `add_file()`. 

In [4]:
import shutil
from llama_index.indices.managed.vectara import VectaraIndex

data_folder = 'temp'
os.makedirs(data_folder, exist_ok=True)

# Create Vectara Index
index = VectaraIndex()

# Upload content for all papers
for paper in papers:
    try:
        paper_fname = paper.download_pdf(data_folder)
    except Exception as e:
        print(f"File {paper_fname} failed to load with error {e}")
        continue
    metadata = {
        'url': paper.pdf_url,
        'title': paper.title,
        'author': str(paper.authors[0]),
        'published': str(paper.published.date())
    }
    index.insert_file(file_path=paper_fname, metadata=metadata)

shutil.rmtree(data_folder)
del papers

File temp/1909.03104v2.Efficient_Sentence_Embedding_using_Discrete_Cosine_Transform.pdf failed to load with error HTTP Error 404: Not Found


Two important things to note here:
1. Vectara processes each file uploaded on the backend, and performs appropriate chunking. So you don't need to apply any local processing, or choose a chunking strategy. If you would like to have more control over how these documents are indexed, such as specifying the maximum number of characters per chunk, you can add the `chunking_strategy` argument to the `insert_file()` function used in the above cell.
2. We have used the fields `url`, `title`, `author`, and `published` as metadata fields (for simplicity, author is the first author if there are multiple). You will need to make sure those fields are defined in your Vectara corpus as [filterable metadata fields](https://docs.vectara.com/docs/learn/metadata-search-filtering/filter-overview) to ensure we can filter by them in query time.

So that's it for upload. 

## Querying with the VectaraIndex
We can now ask questions using the `VectaraIndex` object.

In [5]:
query = "What is sentence embedding?"

In [6]:
query_engine = index.as_query_engine(
    summary_enabled=True, summary_num_results=5,
    summary_response_lang="eng",
    summary_prompt_name="mockingbird-1.0-2024-07-16"
)
res = query_engine.query(query)
print(res.response)

Based on the provided sources, here is a summary that answers the query "What is sentence embedding?":

Sentence embedding is a technique that represents words or sentences in a text in a n-dimensional space, encoding the meaning of the word or sentence [2]. It is a vital technique in practical applications, enabling the extraction of parallel data from extensive web corpora across two languages, enhancing the training of high-quality machine translation and multilingual language models [1]. Sentence embedding is also instrumental for zero-shot cross-lingual retrieval, an essential method for enabling cross-lingual searches on e-commerce platforms like Amazon [1]. In the context of machine learning, sentence embedding is a form of word or sentence representation by learned representations that prepare texts in an understandable format for a machine [2].

Sources: [1], [2]


Note that the response here is fully generated by Vectara. There is no additional LLM involved (or API key you need to setup). The response also includes citations (marked in square brackets), which provide links to references used to generate this response by Vectara. 
<br>
The `res` object includes the actual response to the user query, but also has the citations:

In [7]:
[(inx, n.node.metadata['url']) for inx, n in enumerate(res.source_nodes)]

[(0, 'http://arxiv.org/pdf/2205.15744v2'),
 (1, 'http://arxiv.org/pdf/2206.02690v3'),
 (2, 'http://arxiv.org/pdf/2305.03010v1'),
 (3, 'http://arxiv.org/pdf/2305.03010v1'),
 (4, 'http://arxiv.org/pdf/1904.05542v1'),
 (5, 'http://arxiv.org/pdf/1904.05542v1'),
 (6, 'http://arxiv.org/pdf/1904.05542v1'),
 (7, 'http://arxiv.org/pdf/1904.05542v1'),
 (8, 'http://arxiv.org/pdf/1904.05542v1'),
 (9, 'http://arxiv.org/pdf/1904.05542v1')]

## Using Streaming

You can also stream the Vectara response simply by specifying `streaming=True`:

In [9]:
query_engine = index.as_query_engine(
    summary_enabled=True,
    summary_prompt_name="mockingbird-1.0-2024-07-16",
    streaming=True)

res = query_engine.query(query)

# print streamed output chunk by chunk
res.print_response_stream()

Based on the provided sources, here is a summary that answers the query "What is sentence embedding?":

Sentence embedding is a technique that represents words or sentences in a text in a way that encodes their meaning in n-dimensional space [2]. It is a vital technique in practical applications, enabling the extraction of parallel data from extensive web corpora across two languages, enhancing the training of high-quality machine translation and multilingual language models [1]. Sentence embedding is also instrumental for zero-shot cross-lingual retrieval, an essential method for enabling cross-lingual searches on e-commerce platforms like Amazon [1].

In the context of machine learning, sentence embedding is a form of word or sentence representation by learned representations that prepare texts in an understandable format for a machine [2]. It maps text data into vectors that can be a set of real numbers (a vector), where words or sentences that are closer in the vector space are mor

## Reranking

Vectara supports three types of [reranking](https://docs.vectara.com/docs/api-reference/search-apis/reranking):
1. [Maximal Marginal Relevance](https://docs.vectara.com/docs/learn/mmr-reranker), or MMR, provides a reranking that can promote diversity in results at the cost of relevance.
2. [Slingshot](https://docs.vectara.com/docs/learn/vectara-multi-lingual-reranker) is a mulitilingual reranker that increases the accuracy of retrieved results across 100+ languages and is available to Vectara Scale customers.
3. [User Defined Functions](https://docs.vectara.com/docs/learn/user-defined-function-reranker) allow you to create your own functions for reranking search results, unlocking better retrieval in a wide variety of use cases, such as sorting by recency or price of a product.

 Let's see an example of how to use MMR: We will run the same query but this time we will use MMR where `mmr_diversity_bias=0.3` provides a tradeoff between relevance and diversity (0.0 is full relevance, 1.0 is only diversity):

In [10]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    reranker="mmr",
    rerank_k=50,
    mmr_diversity_bias=0.3,
)
response = query_engine.query(query)
print(response)

Sentence embedding is a technique used in natural language processing to represent sentences as vectors in an n-dimensional space. This representation encodes the meaning of the sentence, allowing for the comparison of semantic similarity between sentences. Sentence embeddings are crucial for various applications, such as machine translation, multilingual language models, and cross-lingual retrieval tasks. They enable efficient processing and understanding of text data by machines, facilitating tasks like document classification and sentiment analysis [1], [3], [5].


In [11]:
[(inx, n.node.metadata['url']) for inx, n in enumerate(response.source_nodes)]

[(0, 'http://arxiv.org/pdf/2205.15744v2'),
 (1, 'http://arxiv.org/pdf/2205.15744v2'),
 (2, 'http://arxiv.org/pdf/2206.02690v3'),
 (3, 'http://arxiv.org/pdf/1904.05542v1'),
 (4, 'http://arxiv.org/pdf/1808.05505v3')]

As you can see, the results are now reranked in a way that provides more diversity instead of maximizing pure relevance. This in turn results in a different set of chunks used to generate the response.

Now let's see an example with a user defined function. We may be interested in getting results that are the most semantically similar to our question, but we also want the most up-to-date information. Thus, we can bias our search results so that the papers that are not only semantically similar but also published more recently are used to answer our query. We can do this by using the available time functions (to see other built-in functions, see the UDF Reranker [documentation](https://docs.vectara.com/docs/learn/user-defined-function-reranker)).

Vectara also supports chain-reranking, which provides a way to chain together multiple reranking methods to achieve better control over the reranking, and combining the strengths of various reranking methods. A great way to use the UDF reranker is in a chain: first the multilingual reranker, followed by the maximal marginal relevance (MMR) reranker, and then a user-defined function, as shown below:

In [12]:
query_engine = index.as_query_engine(
    similarity_top_k = 50,
    reranker="chain",
    rerank_chain=[
        {
            "type": "slingshot"
        },
        {
            "type": "mmr",
            "diversity_bias": 0.3
        },
        {
            "type": "userfn",
            "user_function": "max(0, 5 * get('$.score') - hours(seconds((to_unix_timestamp(now()) - to_unix_timestamp(datetime_parse(get('$.document_metadata.published'), 'yyyy-MM-dd'))))) / 24 / 365)",
            "limit": 5
        }
    ]
)

response = query_engine.query("What innovations have been made to sentence embedding models?")
print(response)

Recent innovations in sentence embedding models include the development of models like HiT, which learns the hierarchical semantic structure in language models and is trained on WordNet to provide unseen subsumptions and hypernyms for words in sentences. Sentence-BERT and gte-base models have also been recognized for their effectiveness in unsupervised text retrieval tasks. Additionally, there is a focus on improving cross-lingual model alignment by incorporating parallel training datasets, which is particularly beneficial for low-resource languages. These advancements highlight the importance of sentence embedding models in applications such as Bi-text Mining, Information Retrieval, and Retrieval Augmented Generation [1], [2].


In [13]:
[(inx, n.node.metadata['published']) for inx, n in enumerate(response.source_nodes)]

[(0, '2024-11-25'),
 (1, '2024-12-04'),
 (2, '2024-11-25'),
 (3, '2024-11-25'),
 (4, '2024-05-30')]

Notice how many of the papers used to generate the final summary were published recently, and they still give us information to generate a relevant response that answers our question.

Also notice how we use a max() function with 0 in our user-defined expression. This is to ensure that all of our reranking scores are non-negative. Additionally, since we multiplied the original score by 10 and its value ranges from 0 to 1, we throw away any search results that are older than 10 years old for generating our final response.

So far we've used Vectara's internal summarization capability, which is the best way for most users.

You can still use Llama-Index's standard VectorStore `as_query_engine()` method, in which case Vectara's summarization won't be used, and you would be using an external LLM (e.g. OpenAI's GPT-4) and a custom prompt from LlamaIndex to generate the summary. For this option just set `summary_enabled=False`

For this functionality, you will need to specify your own OpenAI API key in the environment:

> `os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'`

In [14]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4-turbo", temperature=0)

query_engine = index.as_query_engine(
    similarity_top_k=5,
    summary_enabled=False,
    llm=llm
)
response = query_engine.query(query)
print(response)

Sentence embedding is a technique used in machine learning to represent sentences as vectors in a high-dimensional space. These vectors are designed to capture the semantic meaning of the sentences, allowing machines to understand and process text data more effectively. Sentence embeddings are used in various natural language processing tasks such as machine translation, text classification, and information retrieval. The goal is for sentences with similar meanings to be close to each other in the vector space, despite being composed of different words or structures. This representation facilitates the comparison and manipulation of textual data by algorithms.


## Using Vectara Chat

Vectara now fully supports Chat in its platform, where the chat history is maintained by Vectara and so you don't have to worry about keeping history and integrating it with your RAG pipeline. 

To use it, simply call `as_chat_engine()`.

(Chat mode always uses Vectara's summarization so you don't have to explicitly specify `summary_enabled=True` like before)

In [15]:
ce = index.as_chat_engine()

In [16]:
questions = [
    'What is a sentence embedding model?',
    'What are some known models?',
    'How are they different than token embedding models'
]

for q in questions:
    print(f"Question: {q}\n")
    response = ce.chat(q).response
    print(f"Response: {response}\n")

Question: What is a sentence embedding model?

Response: A sentence embedding model is a type of model that represents a given input sentence as a fixed-dimensional vector, regardless of the sentence's length. These embeddings are particularly useful in various natural language processing (NLP) tasks, such as information retrieval, question answering, and machine translation. Sentence embeddings have shown significant improvements in performance for these tasks compared to static word embeddings, which typically have fewer dimensions [4].

Question: What are some known models?

Response: Some known sentence embedding models include HiT, Sentence-BERT, and gte-base, which are effective in unsupervised text retrieval tasks [1]. Other models include GenSen and DSE, which have been compared across different test sets [4]. Additionally, InferSent, with its two versions based on GloVe and fastText word embeddings, and the Universal Sentence Encoder (USE), which comes in versions based on Dee

Of course streaming works as well with Chat:

In [17]:
ce = index.as_chat_engine(streaming=True)

In [18]:
response = ce.stream_chat("Who is behind SBERT?")
response.print_response_stream()

SBERT, or Sentence-BERT, was developed by Nils Reimers and Iryna Gurevych in 2019. It is based on transformer models like BERT, which was introduced by Devlin et al. in 2018, and fine-tunes these models using a siamese network structure to create sentence embeddings efficiently [3].

# Advanced RAG with Vectara and LLamaIndex

## Agentic RAG

Vectara also has its own package, [vectara-agentic](https://github.com/vectara/py-vectara-agentic), built on top of many features from LlamaIndex to easily implement agentic RAG applications. It allows you to create your own AI assistant with RAG query tools and other custom tools, such as making API calls to retrieve information from financial websites. You can find the full documentation for vectara-agentic [here](https://vectara.github.io/vectara-agentic-docs/).

Let's create a ReAct Agent with a single RAG tool using vectara-agentic (to create a ReAct agent, specify `VECTARA_AGENTIC_AGENT_TYPE` as `"REACT"` in your environment).

Vectara does not yet have an LLM capable of acting as an agent for planning and tool use, so we will need to use another LLM as the driver of the agent resoning.

In this demo, we are using OpenAI's GPT4o. Please make sure you have `OPENAI_API_KEY` defined in your environment or specify another LLM with the corresponding key (for the full list of supported LLMs, check out our [documentation](https://vectara.github.io/vectara-agentic-docs/introduction.html#try-it-yourself) for setting up your environment).

In [19]:
# !pip install -U vectara-agentic

In [20]:
from vectara_agentic.agent import Agent
from IPython.display import display, Markdown

agent = Agent.from_corpus(
    data_description="sentence embeddings",
    assistant_specialty="sentence embeddings research",
    tool_name="ask_embeddings",
    vectara_summary_num_results=5,
    vectara_summarizer="mockingbird-1.0-2024-07-16",
    vectara_reranker="mmr",
    vectara_rerank_k=50,
    verbose=True,
)

response = agent.chat(
    "Tell me about the latest innovations in sentence embedding models."
)

display(Markdown(response))

Failed to set up observer (No module named 'phoenix.otel'), ignoring
> Running step d739bbed-4386-4da0-91ba-7fbe877e38b2. Step input: Tell me about the latest innovations in sentence embedding models.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: ask_embeddings
Action Input: {'query': 'latest innovations in sentence embedding models'}
Observation: 
                    Response: '''The latest innovations in sentence embedding models include HiT, Sentence-BERT, and gte-base models, which have achieved notable breakthroughs in fine-tuning scenarios [2]. However, there is a research gap in computationally efficient direct inference methods for sentence representation [1]. Recent studies have shown that sentence embedding models with both lower alignment and uniformity achieve better performance in general [3]. Additionally, contrastive learning has been applied to sentence embedding models, leading to performance improve

Recent innovations in sentence embedding models include the development of models like HiT, Sentence-BERT, and gte-base, which have made significant advancements in fine-tuning scenarios. However, there is still a need for more computationally efficient direct inference methods for sentence representation. Studies have shown that models with lower alignment and uniformity tend to perform better. Additionally, contrastive learning has been applied to improve performance, as seen in Japanese language models. The use of large language models such as LLaMA and Mistral has also contributed to advancements in this field. For more details, you can refer to the following sources: [1](http://arxiv.org/pdf/2404.03921v2), [2](http://arxiv.org/pdf/2411.16236v1), [3](http://arxiv.org/pdf/2204.10931v1), and [5](http://arxiv.org/pdf/2301.08193v1).

## Summary

In this notebook we've seen various examples for using Vectara with LlamaIndex, which provides the following benefits:
* Vectara provides a complete RAG pipeline, so you don't have to deal with a lot of the details around data ingestion: pre-processing, chunking, embedding, etc. Instead all these steps are handled automatically and efficiently in Vectara. 
* Being a platform, Vectara uses its own internal Embedding model (Boomerang), its own vector storage, and its own LLM (Mockingbird) for summarization, so you don't have to maintain separate API keys and relationships with additional vendors or install other products.
* Vectara is built for large scale GenAI applications, and with the vectara-agentic package, you can easily build and test advanced RAG applications at an enteprise scale.